# Formation of Turing patterns in 2D reaction-diffusion

In this case, we consider a single surface whose edges define a 1D compartment 
- surf - 2D surface
- edge - outer edges of the surface (1D)

This model uses two diffusing species; if one has a much higher diffusion coefficient than the other, then we expect to observe the formation of Turing patterns.


In [ ]:
import os

import dolfin as d
import sympy as sym
import numpy as np

from smart import config, common, mesh, model
from smart.units import unit
from smart.model_assembly import (
    Compartment,
    Parameter,
    Reaction,
    Species,
    SpeciesContainer,
    ParameterContainer,
    CompartmentContainer,
    ReactionContainer,
)
from matplotlib import pyplot as plt

First, we define the various units for the inputs

In [ ]:
# Aliases - base units
uM = unit.uM
um = unit.um
molecule = unit.molecule
sec = unit.sec
dimensionless = unit.dimensionless
# Aliases - units used in model
D_unit = um**2 / sec
flux_unit = molecule / (um**2 * sec)
surf_unit = molecule / um**2
edge_unit = molecule / um

Next we generate the model.

In [ ]:
# define dimensions of domain
x_size = 10
y_size = 10

# =============================================================================================
# Compartments
# =============================================================================================
# name, topological dimensionality, length scale units, marker value
surf = Compartment("surf", 2, um, 1)
edge = Compartment("edge", 1, um, 3)
cc = CompartmentContainer()
cc.add([surf, edge])

# =============================================================================================
# Species
# =============================================================================================
# name, initial concentration, concentration units, diffusion, diffusion units, compartment
# Ainit = f"10*exp(-((x-{a}/4.0)**2+(y-{b}/2.0)**2)/.1)"
# Binit = f"exp(-((x-3.0*{a}/4.0)**2+(y-{b}/2.0)**2)/.1)"
D_ref = 1.0 # reference diffusion coeff
d_ratio = 20.0 # ratio between diffusion coefficients
A = Species("A", 1.0, surf_unit, D_ref, D_unit, "surf") # activator
B = Species("B", f"0.5*({x_size}-x)", surf_unit, d_ratio*D_ref, D_unit, "surf") # inhibitor
sc = SpeciesContainer()
sc.add([A, B])

# =============================================================================================
# Parameters and Reactions
# =============================================================================================
# Describing the Schnakenberg system here
g = Parameter("g", 50.0, dimensionless / sec)
a = Parameter("a", 0.2, dimensionless)
b = Parameter("b", 2.0, dimensionless)
# D_A = Parameter("D_A", D_ref, D_unit)
# k1 = Parameter("k1", 1.0, surf_unit/sec)
# L = Parameter("L", 1.0, um)
cref = Parameter("cref", 1.0, surf_unit)

# Production of A
r1 = Reaction("r1", [], ["A"], 
              param_map={"a": "a", "g": "g", "cref":"cref"}, 
              eqn_f_str="cref*g*(a + (A/cref)**2 * (B/cref))", 
              species_map={"A": "A", "B": "B"})

# Degradation of A
r2 = Reaction("r2", ["A"], [], 
              param_map={"g": "g", "cref":"cref"}, 
              eqn_f_str="cref*g*A/cref", 
              species_map={"A": "A"})

# Production of B
r3 = Reaction("r3", [], ["B"], 
              param_map={"g":"g","b": "b", "cref":"cref"}, 
              eqn_f_str="cref*g*b")

# Degradation of B
r4 = Reaction("r4", ["B"], [], 
              param_map={"g": "g", "cref":"cref"}, 
              eqn_f_str="cref*g* (A/cref)**2 * (B/cref)", 
              species_map={"A": "A", "B": "B"})

# # B-dependent inhibition of A
# fB = Parameter("fB", -4, 1/sec)
# r1 = Reaction("r1", [], ["A"], param_map={"fB": "fB"}, 
#               eqn_f_str="fB*B", species_map={"B": "B"})

# # A self activation
# fA = Parameter("fA", 1, 1/sec)
# r2 = Reaction("r2", [], ["A"], param_map={"fA": "fA"}, 
#               eqn_f_str="fA*A", species_map={"A": "A"})

# # B self inhibition
# gB = Parameter("gB", -2, 1/sec)
# r3 = Reaction("r3", [], ["B"], param_map={"gB": "gB"}, 
#               eqn_f_str="gB*B", species_map={"B": "B"})

# # A-dependent activation of B
# gA = Parameter("gA", 1, 1/sec)
# r4 = Reaction("r4", [], ["B"], param_map={"gA": "gA"}, 
#               eqn_f_str="gA*A", species_map={"A": "A"})

# A boundary fluxes
AFlux = Parameter("AFlux", 0, edge_unit/sec)
r5 = Reaction("r5", ["A"], [], param_map={"AFlux": "AFlux"},
              eqn_f_str="AFlux", explicit_restriction_to_domain="edge")

# B boundary fluxes
BFlux = Parameter("BFlux", 0, edge_unit/sec)
r6 = Reaction("r6", ["B"], [], param_map={"BFlux": "BFlux"},
              eqn_f_str="BFlux", explicit_restriction_to_domain="edge")

pc =ParameterContainer()
pc.add([a, b, g, cref, AFlux, BFlux])
rc = ReactionContainer()
rc.add([r1, r2, r3, r4, r5, r6])

We load the model generated above, and load in the mesh we will use in this example, iterating over 10 different values of cell radius, log-spaced between 1 and 10.

In [ ]:
# Create mesh
m = 100; n = int(x_size/y_size)*m
rect_mesh = d.RectangleMesh(d.Point(0.0, 0.0), d.Point(x_size, y_size), n, m)
mf2 = d.MeshFunction("size_t", rect_mesh, 2, 1)
# for f in d.faces(rect_mesh):
#     x_vals = np.zeros(3)
#     idx = 0
#     for vertex in d.vertices(f):
#         x_vals[idx] = vertex.point().array()[0]
#         idx = idx+1
#     if np.mean(x_vals) > a/2.0:
#         mf2[f] = 2
mf1 = d.MeshFunction("size_t", rect_mesh, 1, 0)
for e in d.edges(rect_mesh):
    x_vals = np.zeros(2)
    y_vals = np.zeros(2)
    idx = 0
    for vertex in d.vertices(e):
        x_vals[idx] = vertex.point().array()[0]
        y_vals[idx] = vertex.point().array()[1]
        idx = idx+1
    if np.isclose(np.mean(x_vals), 0.) or np.isclose(np.mean(x_vals),x_size)\
        or np.isclose(np.mean(y_vals),0) or np.isclose(np.mean(y_vals),y_size):
        mf1[e] = 3
# Write mesh and meshfunctions to file
os.makedirs(f"rect_mesh", exist_ok=True)
filename = f"rect_mesh/rect_mesh"
hdf5 = d.HDF5File(rect_mesh.mpi_comm(), filename + ".h5", "w")
hdf5.write(rect_mesh, "/mesh")
hdf5.write(mf2, "/mf2")
hdf5.write(mf1, "/mf1")
d.File(f"{filename}_mf2.pvd") << mf2
d.File(f"{filename}_mf1.pvd") << mf1

# # Define solvers
parent_mesh = mesh.ParentMesh(
    mesh_filename=f"{filename}.h5",
    mesh_filetype="hdf5",
    name="parent_mesh",
)
configCur = config.Config()
modelCur = model.Model(pc, sc, cc, rc, configCur, parent_mesh)
configCur.solver.update(
    {
        "final_t": 1.0,
        "initial_dt": 0.001,
        "time_precision": 6,
        "use_snes": True,
        "print_assembly": False,
    }
)

modelCur.initialize(initialize_solver=False)
modelCur.initialize_discrete_variational_problem_and_solver()
# Write initial condition(s) to file
results = dict()
os.makedirs(f"resultsRect", exist_ok=True)
for species_name, species in modelCur.sc.items:
    results[species_name] = d.XDMFFile(
        modelCur.mpi_comm_world, f"resultsRect/{species_name}.xdmf"
    )
    results[species_name].parameters["flush_output"] = True
    results[species_name].write(modelCur.sc[species_name].u["u"], modelCur.t)

# Solve
while True:
    # Solve the system
    modelCur.monolithic_solve()
    # Save results for post processing
    for species_name, species in modelCur.sc.items:
        results[species_name].write(modelCur.sc[species_name].u["u"], modelCur.t)
    # End if we've passed the final time
    if modelCur.t >= modelCur.final_t:
        break

# compute steady state solution at the end of each run
dx = d.Measure("dx",domain = modelCur.cc['surf'].dolfin_mesh)
int_val = d.assemble(modelCur.sc['A'].u['u']*dx)
SA = d.assemble(1.0*dx)
ss_val = int_val / SA

In [ ]:
# %matplotlib inline
# plt.plot(radiusVec, ss_vec, 'ro')
# radiusTest = np.logspace(0,1,100)
# thieleMod = radiusTest / 1.0
# k_kin = 50
# k_p = 10
# cT = 1
# D = 10
# C1 = k_kin*cT*radiusTest**2/((3*D*(np.sqrt(k_p/D)-(1/radiusTest)) + k_kin*radiusTest)*np.exp(thieleMod) +
#         (3*D*(np.sqrt(k_p/D)+(1/radiusTest))-k_kin*radiusTest)*np.exp(-thieleMod))
# cA = (6*C1/radiusTest)*(np.cosh(thieleMod)/thieleMod - np.sinh(thieleMod)/thieleMod**2)
# plt.plot(radiusTest, cA)
# plt.xlabel("Cell radius (μm)")
# plt.ylabel("Steady state concentration (μM)")
# plt.legend(("SMART simulation", "Analytical solution"))